The following blocks of code give an overview of this toy example.

"Ramen" have 6 attributes: label, brand, country of origin, style, rating, and price. The following block shows how we
set up a pipeline to generate recommendations for a fake 'user' profile.

First we'll look at an example of loading up rdf data and running a pipeline to generate and rank 
recommended ramens for a user.

In [21]:
from examples.ramen_rec import *
from frex.stores import LocalGraph
from rdflib import URIRef

# the files we will load containing ramen ratings, fake 'user' data, and vector representation of the ramens.
data_files = ((RamenUtils.DATA_DIR / "ramen-ratings.ttl").resolve(),)
user_files = ((RamenUtils.DATA_DIR / "ramen-users.ttl").resolve(),)
vector_file = RamenUtils.DATA_DIR / "ramen-vectors.pkl"

# set up a Graph to load and store the ramen data
ramen_graph = LocalGraph(file_paths=data_files)
# set up the query service to access ramen data from the graph
ramen_q = GraphRamenQueryService(queryable=ramen_graph)

# similarly, set up the graph and query service for user data
user_graph = LocalGraph(file_paths=user_files)
ramen_eater_q = GraphRamenEaterQueryService(queryable=user_graph)

In [22]:
# our dummy user (ramen eater) who we will get recommendations for
ramen_eater_uri = URIRef('http://www.frex.com/examples/USR01')
target_ramen_eater = ramen_eater_q.get_ramen_eater_by_uri(
    ramen_eater_uri=ramen_eater_uri
)
print(f"Demo retrieve recommendations for ramens eater {target_ramen_eater.uri}")

Demo retrieve recommendations for ramens eater http://www.frex.com/examples/USR01


In [23]:
# set up a pipeline that will generate and score candidate Ramens for this user
ramen_rec_pipe = RecommendForEaterPipeline(
    vector_file=vector_file, ramen_query_service=ramen_q
)

In [24]:
# pass in the user context and run the pipeline
output_candidates = tuple(ramen_rec_pipe(context=RamenEaterContext(ramen_eater_profile=target_ramen_eater)))

# show some of the candidate contents
for candidate in output_candidates[:2]:
    do = candidate.domain_object
    print(f'candidate ramen label: {do.label}, style: {do.style}, rating: {do.rating}')

candidate ramen label: Science Noodle (X'Mas Edition), style: Pack, rating: 5.0
candidate ramen label: Dan Dan Noodle, style: Pack, rating: 5.0


In [25]:
print('Example explanations applied to a candidate output by the pipeline: ')

# we can also look at the explanations applied to the candidate, which gives us some insight into what kind of filtering
# and scoring went on in the pipeline.
for expl in output_candidates[0].applied_explanations:
    print(expl.explanation_string)

Example explanations applied to a candidate output by the pipeline: 
This ramen is identified as being similar to all of the user's favorite ramens.
This ramen is not from a country that is prohibited by the eater.
This ramen has a high rating score.
This ramen is from not a brand that the user likes.
This ramen is a style that the user likes.
This ramen is from a country that the user likes.


We can also use constraint solving to produce a combination of highly recommended items.

Let's consider an example of making a meal plan for 2 days, eating 3 ramens each day (we'll assume health and dignity
are not important considerations for this user).

For this meal plan, we want to choose Ramens that are highly scored by the recommendation pipeline for our user.
Additionally, we want to apply some constraints:
- The total price of ramens eaten for a given day is <= $7.00
- The total price of all ramens eaten in the meal plan (2 days x 3 ramens) is <= $13.00
- Ratings are important to us, so the combined ratings of ramens eaten each day must be >= 7

The following block of code shows how to set up and solve these constraints.

In [26]:
from frex.utils import ConstraintSolver, ConstraintType

# this will initialize a constraint solver,
# set it to 2 sections (2 days),
# require each section to have 3 items (3 ramens per day),
# create a price constraint for each day,
# create a rating constraint for each day,
# and create a price constraint for all items
solution = ConstraintSolver().\
    set_candidates(candidates=output_candidates)\
    .set_sections(num_sections=2)\
    .set_items_per_section(count=3)\
    .add_section_constraint(attribute_name='price', constraint_type=ConstraintType.LEQ, constraint_val=7.0)\
    .add_section_constraint(attribute_name='rating', constraint_type=ConstraintType.GEQ, constraint_val=7)\
    .add_overall_constraint(attribute_name='price', constraint_type=ConstraintType.LEQ, constraint_val=13.0)\
    .solve()

In [27]:
# exploring the solution. This is based on rating scores acquired from the recommender pipeline, so
# they don't really mean a whole lot in the context of this toy example besides high score = better
print(f'Total score of the obtained solution: {round(solution.overall_score, 2)}')

print('Day 1:')
print(f'total price of ramens for day 1: {round(solution.sections[0].section_attribute_values["price"], 2)}')
print('Ramen names, ratings, and scores in day 1:')
for candidate in solution.sections[0].section_candidates:
    do = candidate.domain_object
    print(f'name: {do.label}, rating: {do.rating}, price: {do.price}, recommendation score: {round(candidate.total_score, 2)}')
print('Day 2:')
print(f'total price of ramens for day 2: {round(solution.sections[1].section_attribute_values["price"], 2)}')
print('Ramen names, ratings, and scores in day 2:')
for candidate in solution.sections[1].section_candidates:
    do = candidate.domain_object
    print(f'name: {do.label}, rating: {do.rating}, price: {do.price}, recommendation score: {round(candidate.total_score, 2)}')

Total score of the obtained solution: 18.200000000000003
Day 1:
total price of ramens for day 1: 6.640000000000001
Ramen names, ratings, and scores in day 1:
name: Baseball Snack Noodle, rating: 5.0, recommendation score: 3.1
name: Traditional Shallot & Onion Oil Noodle, rating: 5.0, recommendation score: 3.1
name: Little Prince(ss) Brand Snack Noodles Artificial Mexican Pizza Flavor, rating: 4.0, recommendation score: 2.9000000000000004
Day 2:
total price of ramens for day 2: 5.930000000000001
Ramen names, ratings, and scores in day 2:
name: Science Noodle (X'Mas Edition), rating: 5.0, recommendation score: 3.1
name: Scallion Oil & Soy Sauce Noodle, rating: 5.0, recommendation score: 3.1
name: Sun Dried Noodle - Fruity Soy Bean Paste, rating: 4.0, recommendation score: 2.9000000000000004


We can compare this to just using the first few candidates that we got from the recommendation if pipeline. We'll see
that some of the first candidates from the pipeline aren't in our solution that was based on constraints since they have
a relatively high price.

(These recommendations also have a lot of ties in the scores, since the methods used to score them are extremely simple)

In [29]:
for candidate in output_candidates[:4]:
    do = candidate.domain_object
    print(f'name: {do.label}, rating: {do.rating}, price: {do.price}, recommendation score: {round(candidate.total_score, 2)}')

name: Science Noodle (X'Mas Edition), rating: 5.0, price: 1.93, recommendation score: 3.1
name: Dan Dan Noodle, rating: 5.0, price: 2.87, recommendation score: 3.1
name: Sichuan Spices Flavor Noodle, rating: 5.0, price: 4.16, recommendation score: 3.1
name: Sichuan Spicy Flavor, rating: 5.0, price: 3.91, recommendation score: 3.1
